# Visualize prompt results

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
import plotly.graph_objects as go

# Parameter für die Auswahl der Vergleichs- und Hovertext-Spalten
comparison_column = 'Spätestmögliches Datum'  # Optionen: 'Datum', 'Nummer', 'Frühstmögliches Datum', 'Spätestmögliches Datum', 'Seitenzahl'
hovertext_column = 'Verglichene Werte'  # Optionen: 'Datierung', 'Verglichene Werte'

# Datei einlesen (ersetze 'deine_datei.xlsx' mit dem tatsächlichen Dateinamen)
df = pd.read_csv('gold_vs_results.tsv', sep="\t")

# Nur die relevanten Spalten auswählen
df = df[['Prompt', 'Nummer', 'Datum', 'Frühstmögliches Datum', 'Spätestmögliches Datum', 'Datierung', 'Seitenzahl']]

# Zeilennummer innerhalb jedes Prompts hinzufügen
df['Zeile'] = df.groupby('Prompt').cumcount()

# Pivot-Tabelle erstellen, um die Daten für den Vergleich vorzubereiten
pivot_table = df.pivot(index='Zeile', columns='Prompt', values=['Datum', 'Nummer', 'Frühstmögliches Datum', 'Spätestmögliches Datum', 'Datierung', 'Seitenzahl'])

# Paare von Prompts ermitteln
prompt_pairs = list(combinations(pivot_table[comparison_column].columns, 2))

# Vergleichsmatrix initialisieren
comparison_matrix = pd.DataFrame(np.nan, index=pivot_table.index, columns=[f'{p1} vs {p2}' for p1, p2 in prompt_pairs])
hover_text_matrix = pd.DataFrame('', index=pivot_table.index, columns=[f'{p1} vs {p2}' for p1, p2 in prompt_pairs])

# Paarweiser Vergleich und Hovertext
for p1, p2 in prompt_pairs[0:20]:
    comparison_matrix[f'{p1} vs {p2}'] = (pivot_table[comparison_column][p1] == pivot_table[comparison_column][p2]).astype(int)
    
    if hovertext_column == 'Datierung':
        hover_text_matrix[f'{p1} vs {p2}'] = pivot_table['Datierung'][p1]
    elif hovertext_column == 'Verglichene Werte':
        hover_text_matrix[f'{p1} vs {p2}'] = (
            pivot_table['Nummer'][p1].astype(str) + " " + pivot_table['Datierung'][p1].astype(str) + "<br>a: "+ pivot_table[comparison_column][p1].astype(str) + "<br>b: " + pivot_table[comparison_column][p2].astype(str)
        )

# Die "Nummer"-Labels als X-Achse festlegen
x_labels = pivot_table['Nummer'][pivot_table['Nummer'].columns[0]]

# Erstellen der Heatmap mit Plotly
heatmap = go.Heatmap(
    z=comparison_matrix.T.values,  # Transponiere die Vergleichsmatrix
    x=x_labels,
    y=comparison_matrix.T.index[0:20],  # Die Vergleichspaare als Y-Achse
    colorscale='Blues_r',  # Verwende die 'Blues' Farbskala für hellere Farben
    showscale=False,
    text=hover_text_matrix.T.values,  # Transponiere die Hover-Text-Matrix
    hoverinfo='text',
    zmin=0,  # Mindestwert für die Farbskala
    zmax=1,  # Höchstwert für die Farbskala
    xgap=1,  # Abstand zwischen den Zellen in X-Richtung
    ygap=1   # Abstand zwischen den Zellen in Y-Richtung
)

# Layout für die Plotly-Grafik definieren
layout = go.Layout(
    title=f'Vergleich der Spalte "{comparison_column}"',
    xaxis=dict(
        # title='Nummer',
        showgrid=False,  # Gitternetzlinien auf der X-Achse ausblenden
        zeroline=False,  # Null-Linie auf der X-Achse ausblenden
    ),
    yaxis=dict(
        # title='Vergleichspaare',
        showgrid=False,  # Gitternetzlinien auf der Y-Achse ausblenden
        zeroline=False,  # Null-Linie auf der Y-Achse ausblenden
        autorange="reversed"
    ),
#    width=700,
#    height=700
    width=2000,
    height=1000
)

# Figure erstellen und anzeigen
fig = go.Figure(data=[heatmap], layout=layout)

fig.update_layout(
#    plot_bgcolor='black',
    font=dict(
        size=15,  # Set the font size here
        color='black'
    )
)
fig.update_yaxes(labelalias={'Gold vs I-4o-01':'4o-01',
                             'Gold vs I-4o-02':'4o-02',
                             'Gold vs I-4o-03':'4o-03',
                             'Gold vs I-4o-04':'4o-04',
                             'Gold vs I-4o-05':'4o-05',
                             'Gold vs I-4o-mini-01':'4o-mini-01',
                             'Gold vs I-4o-mini-02':'4o-mini-02',
                             'Gold vs I-4o-mini-03':'4o-mini-03',
                             'Gold vs I-4o-mini-04':'4o-mini-04',
                             'Gold vs I-4o-mini-05':'4o-mini-05',
                             'Gold vs II-4o-01':'4o-01',
                             'Gold vs II-4o-02':'4o-02',
                             'Gold vs II-4o-03':'4o-03',
                             'Gold vs II-4o-04':'4o-04',
                             'Gold vs II-4o-05':'4o-05',
                             'Gold vs II-4o-mini-01':'4o-mini-01',
                             'Gold vs II-4o-mini-02':'4o-mini-02',
                             'Gold vs II-4o-mini-03':'4o-mini-03',
                             'Gold vs II-4o-mini-04':'4o-mini-04',
                             'Gold vs II-4o-mini-05':'4o-mini-05',      
    })
fig.show()
